# Решатель для тестов Богатова

Для работы программы необходимо удаленное подключение к серверу ФТФ КубГУ через **xfreerdp**. Программа работает на OCR с помощью **PyTesseract**, преобразования изображений с помощью **OpenCV** и **pyautogui** для перемещения курсора.

* Тест берет вопросы из пула в произвольном порядке, поэтому нам нужно автоматически распознавать текст вопроса.

In [1]:
import time
import pyautogui
import cv2
import mss
import numpy
import pytesseract
import csv 

* Множество **WHITELIST** хранит допустимые для считывания с помощью OCR символы в вопросе теста.
* **MON** хранит координаты считываемой области.
* **BUTTONS** хранит координаты кнопок в UI теста.

In [8]:
WHITELIST = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ1234567890 ')
MON = {'top': 92, 'left': 1, 'width': 845, 'height': 154}
BUTTONS = {
    'bn': {'c1': 450, 'c2': 90},
    'b1': {'c1': 20, 'c2': 350},
    'b2': {'c1': 450, 'c2': 350},
    'b3': {'c1': 20, 'c2': 500},
    'b4': {'c1': 450, 'c2': 500},
    'by': {'c1': 470, 'c2': 450},
}

In [19]:
def click(cs):
    pyautogui.moveTo(cs['c1'], cs['c2'])
    pyautogui.click()

    
def to_csv(data, filename):
    file = open(filename, 'w+', newline ='') 
    with file:     
        write = csv.writer(file) 
        write.writerows(data)
        

def read_questions(filename, test_length):
    data = []
    with mss.mss() as sct:
        for i in range(test_length):
            im = numpy.asarray(sct.grab(MON))
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

            text = pytesseract.image_to_string(im, lang='rus')
            text = ''.join(filter(WHITELIST.__contains__, text))
            
            data.append([text])
            click(BUTTONS['b1'])
            time.sleep(0.1)
            click(BUTTONS['bn'])
            time.sleep(0.1)
            click(BUTTONS['by'])
            time.sleep(0.5)
    to_csv(data, filename)
    

def csv_reader(csv_path, unique_length):
    answers = dict()
    with open(csv_path, "r") as f_obj:
        reader = csv.reader(f_obj)
        for row in reader:
            answers[row[0].lower()[:unique_length]] = row[1]
    return answers


def resolve_test(path, test_length, unique_length):
    answers = csv_reader(path, unique_length)
    with mss.mss() as sct:
        for i in range(test_length):
            im = numpy.asarray(sct.grab(MON))
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

            text = pytesseract.image_to_string(im, lang='rus')
            text = ''.join(filter(WHITELIST.__contains__, text)).lower()[:unique_length]
            
            answer = answers[text]
            
            if answer == '1':
                click(BUTTONS['b1'])
            if answer == '2':
                click(BUTTONS['b2'])
            if answer == '3':
                click(BUTTONS['b3'])
            if answer == '4':
                click(BUTTONS['b4'])
            time.sleep(0.1)
            click(BUTTONS['bn'])
            time.sleep(0.1)
            click(BUTTONS['by'])
            time.sleep(0.5)

### Важно! Запустить один раз для считывания вопросов, если необходимо

* Первый параметр - путь к файлу для сохранения вопросов.
* Второй параметр - длина теста (число вопросов).

In [ ]:
read_questions('data/test.csv', 50)

### Запускать при каждом решении теста

* Первый параметр - путь к файлу для сохранения вопросов.
* Второй параметр - длина теста (число вопросов).
* Третий параметр - длина текста вопроса, по которому можно уникально идентифицировать вопрос.

In [ ]:
resolve_test('data/test.csv', 50, 50)